In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Differential Log Fold Change (LFC) for PBMC data.

In [ ]:
from IPython import display
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from tensorflow import keras
import tensorflow as tf
import tensorflow_probability as tfp
import pickle
import os

Root_Folder = "/content/drive/MyDrive"
data = 'pbmc'
data_parent_folder = "/Data"
code_parent_folder = "/scMVI"

os.chdir('/content/drive/My Drive/' + code_parent_folder +'/Code')

Rna_train = pd.read_pickle(Root_Folder + data_parent_folder + "/Data_ATAC/Rna_train_"+data+".pickle")
Rna_nrm_train = pd.read_pickle(Root_Folder + data_parent_folder + "/Data_ATAC/Nrm_rna_train_"+data+".pickle")
Rna_test = pd.read_pickle(Root_Folder + data_parent_folder + "/Data_ATAC/Rna_test_"+data+".pickle")
Rna_nrm_test = pd.read_pickle(Root_Folder + data_parent_folder + "/Data_ATAC/Nrm_rna_test_"+data+".pickle")

In [ ]:
Rna_tr = tf.convert_to_tensor(Rna_train,dtype=tf.float32)
Rna_tst = tf.convert_to_tensor(Rna_test,dtype=tf.float32)

Rna_nrm_tr = tf.convert_to_tensor(Rna_nrm_train,dtype=tf.float32)
Rna_nrm_tst = tf.convert_to_tensor(Rna_nrm_test,dtype=tf.float32)

In [ ]:
#Load Metadata
Rna = pd.concat([Rna_train,Rna_test],axis=0)
if data == "pbmc":
    pbmc_QC = pd.read_csv(Root_Folder + data_parent_folder + "/Data_ATAC/Srt_annot.csv",'\t',index_col=[0])
    pbmc_QC = pbmc_QC.loc[Rna.index]
    print(np.unique(list(pbmc_QC['seurat_annotation'])))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


['CD14 Mono' 'CD16 Mono' 'CD4 Naive' 'CD4 TCM' 'CD4 TEM' 'CD8 Naive'
 'CD8 TEM_1' 'CD8 TEM_2' 'HSPC' 'Intermediate B' 'MAIT' 'Memory B' 'NK'
 'Naive B' 'Plasma' 'Treg' 'cDC' 'gdT' 'pDC']


In [ ]:
Rna_comb = tf.concat([Rna_tr,Rna_tst],axis=0)
Rna_nrm_comb = tf.concat([Rna_nrm_tr,Rna_nrm_tst],axis=0)
xxxx = tf.divide(Rna_comb,tf.math.exp(Rna_nrm_comb))

In [ ]:
#Functions for estimating differential directions and stats for experiments outlined in the main chapter 
def index_creation(meta_data,group_by,group_1,smps_group_1,group_2,smps_group_2):


                        cell_num = meta_data.shape[0]
                        group_index_1 = np.zeros((cell_num,))
                        group_index_2 = np.zeros((cell_num,))

                        ind_1 = np.where(meta_data[group_by]==group_1)[0]
                        ind_2 = np.where(meta_data[group_by]==group_2)[0]


                        if np.logical_or(len(ind_1)<smps_group_1,len(ind_2)<smps_group_2):

                                    raise ValueError('samples for groups 1 and 2 cannot be more than cells in groups 1 and 2')        



                        perm_ind_1 = ind_1[np.random.permutation(len(ind_1))[:smps_group_1]]
                        perm_ind_2 = ind_2[np.random.permutation(len(ind_2))[:smps_group_2]]

                        group_index_1[perm_ind_1]=1
                        group_index_2[perm_ind_2]=1

                        group_index_1 = tf.squeeze(tf.where(group_index_1==1))
                        group_index_2 = tf.squeeze(tf.where(group_index_2==1))

                        return group_index_1, group_index_2


def create_exp(metadata,grp_1,smps_1,grp_2,smps_2):
  
        smps_group_1 =[smps_1,int(np.round(smps_1*0.2)),smps_1,int(np.round(smps_1*0.2)),int(np.round(smps_1*0.5)),int(np.round(smps_1*0.5)),int(np.round(smps_1*0.2))]
        smps_group_2 =[smps_2,smps_2,int(np.round(smps_2*0.2)),int(np.round(smps_2*0.5)),int(np.round(smps_2*0.2)),int(np.round(smps_2*0.5)),int(np.round(smps_2*0.2))]

        eps=0.0001
        col_all = []
        for jj in range(5):
              group_index_11,group_index_21 = index_creation(metadata,'seurat_annotation',grp_1,smps_group_1[0],grp_2,smps_group_2[0])
              group_index_12,group_index_22 = index_creation(metadata,'seurat_annotation',grp_1,smps_group_1[1],grp_2,smps_group_2[1])
              group_index_13,group_index_23 = index_creation(metadata,'seurat_annotation',grp_1,smps_group_1[2],grp_2,smps_group_2[2])
              group_index_14,group_index_24 = index_creation(metadata,'seurat_annotation',grp_1,smps_group_1[3],grp_2,smps_group_2[3])
              group_index_15,group_index_25 = index_creation(metadata,'seurat_annotation',grp_1,smps_group_1[4],grp_2,smps_group_2[4])
              group_index_16,group_index_26 = index_creation(metadata,'seurat_annotation',grp_1,smps_group_1[5],grp_2,smps_group_2[5])
              group_index_17,group_index_27 = index_creation(metadata,'seurat_annotation',grp_1,smps_group_1[6],grp_2,smps_group_2[6])

              a1 = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_11),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_21),axis=0))
              a2 = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_12),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_22),axis=0))
              a3 = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_13),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_23),axis=0))
              a4 = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_14),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_24),axis=0))
              a5 = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_15),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_25),axis=0))
              a6 = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_16),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_26),axis=0))
              a7 = tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_17),axis=0))-tf.math.log(eps+tf.reduce_mean(tf.gather(xxxx,group_index_27),axis=0))              

              col_all.append([a1,a2,a3,a4,a5,a6,a7])
              print(100*(1+jj)/5.0)

        return col_all
      
def collect_stats(x):

        x_med = tf.squeeze(x)
        x_med_mn = tf.reduce_mean(tf.reduce_mean(x_med,axis=1),axis=0)[tf.newaxis,:]
        x_med_std = tf.math.sqrt(tf.reduce_mean(tfp.stats.variance(x_med,sample_axis=1),axis=0))[tf.newaxis,:]

        x_med_rng = tf.reduce_mean(tf.reduce_max(x_med,axis=1) -tf.reduce_min(x_med,axis=1),axis=0)[tf.newaxis,:]

        return x_med_mn,x_med_std,x_med_rng


In [ ]:
print(pbmc_QC.value_counts())

seurat_annotation
CD14 Mono            2653
CD4 Naive            1335
CD8 Naive            1324
CD4 TCM              1100
CD16 Mono             490
NK                    426
Memory B              355
Intermediate B        337
CD8 TEM_2             334
CD8 TEM_1             307
CD4 TEM               291
cDC                   191
Treg                  150
gdT                   144
Naive B               137
MAIT                  131
pDC                   104
HSPC                   25
Plasma                 16
dtype: int64


In [ ]:
#Estimates differential directions
grp_1 = 'CD16 Mono' 
smps_1 = 490
grp_2 = 'CD14 Mono'
smps_2 = 2653

col_all_1 = create_exp(pbmc_QC,grp_1,smps_1,grp_2,smps_2)

20.0
40.0
60.0
80.0
100.0


In [ ]:
grp_1 = 'CD8 Naive'
smps_1 = 1324
grp_2 = 'Intermediate B'
smps_2 = 337

col_all_2 = create_exp(pbmc_QC,grp_1,smps_1,grp_2,smps_2)

20.0
40.0
60.0
80.0
100.0


In [ ]:
grp_1 = 'CD4 TEM'
smps_1 = 291
grp_2 = 'CD4 TCM'
smps_2 = 1100

col_all_3 = create_exp(pbmc_QC,grp_1,smps_1,grp_2,smps_2)

20.0
40.0
60.0
80.0
100.0


In [ ]:
cl_1_med,_,cl_1_med_rng = collect_stats(col_all_1)
cl_2_med,_,cl_2_med_rng = collect_stats(col_all_2)
cl_3_med,_,cl_3_med_rng = collect_stats(col_all_3)

In [ ]:
rna_ind = []
for ii in range(5000):
    rna_ind.append("rna_ft"+str(ii))

In [ ]:
med_1_rng = pd.DataFrame(np.squeeze(cl_1_med_rng),index=rna_ind)
med_2_rng = pd.DataFrame(np.squeeze(cl_2_med_rng),index=rna_ind)
med_3_rng = pd.DataFrame(np.squeeze(cl_3_med_rng),index=rna_ind)

med_1 = pd.DataFrame(np.squeeze(cl_1_med),index=rna_ind)
med_2 = pd.DataFrame(np.squeeze(cl_2_med),index=rna_ind)
med_3 = pd.DataFrame(np.squeeze(cl_3_med),index=rna_ind)

In [ ]:
print(med_1.iloc[995:1004,:])
print(med_1_rng.iloc[995:1004,:])

                   0
rna_ft995   1.025389
rna_ft996   1.792918
rna_ft997  -0.396042
rna_ft998  -1.762736
rna_ft999   0.000000
rna_ft1000 -1.655847
rna_ft1001  0.000000
rna_ft1002  0.209888
rna_ft1003  0.116154
                   0
rna_ft995   4.403353
rna_ft996   0.451669
rna_ft997   2.286999
rna_ft998   0.574932
rna_ft999   0.000000
rna_ft1000  0.422965
rna_ft1001  0.000000
rna_ft1002  1.303073
rna_ft1003  2.967103


In [ ]:
if os.path.isdir(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/PBMC") == False:
      os.mkdir(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/PBMC")

med_1_rng.to_csv(Root_Folder+ code_parent_folder + "/scMVI_DDLFC_meta/PBMC/med_1_rng_Differentia_Testing.csv",header=True,sep='\t')
med_2_rng.to_csv(Root_Folder+ code_parent_folder + "/scMVI_DDLFC_meta/PBMC/med_2_rng_Differentia_Testing.csv",header=True,sep='\t')
med_3_rng.to_csv(Root_Folder+ code_parent_folder + "/scMVI_DDLFC_meta/PBMC/med_3_rng_Differentia_Testing.csv",header=True,sep='\t')

med_1.to_csv(Root_Folder+ code_parent_folder + "/scMVI_DDLFC_meta/PBMC/Diff_Dir_1_Differentia_Testing.csv",header=True,sep='\t')
med_2.to_csv(Root_Folder+ code_parent_folder + "/scMVI_DDLFC_meta/PBMC/Diff_Dir_2_Differentia_Testing.csv",header=True,sep='\t')
med_3.to_csv(Root_Folder+ code_parent_folder + "/scMVI_DDLFC_meta/PBMC/Diff_Dir_3_Differentia_Testing.csv",header=True,sep='\t')